In [1]:
# import sys
# from google.colab import drive

# drive.mount('/content/gdrive', force_remount=True)
# SRC_PATH = '/content/gdrive/MyDrive/MP FEB/Colab'
# sys.path.append(SRC_PATH)

# !pip install wandb -qqq
# import wandb
# wandb.login()

# wandb.init(
#     # Set the project where this run will be logged
#     project="baseline_poc", 
#     # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
#     #name="experiment 1"
#     # Track hyperparameters and run metadata
#     #config={
#       #"learning_rate": 0.02,
#       #"architecture": "CNN",
#       #"dataset": "CIFAR-100",
#       #"epochs": 10,}
#     )

In [2]:
%load_ext autoreload
%autoreload 2

import dask.dataframe as dd
import os
import numpy as np
import pandas as pd
import torch

import matplotlib.pyplot as plt
%matplotlib inline

DEVICE = 'cuda'

# 1. Import Dataset

In [3]:
data_folder = str(os.getcwd()) + "\data"

blumenstock_types = {'LOAN_SEQUENCE_NUMBER': str, 'INT_RATE': float, 'ORIG_UPB': float, 'FICO_SCORE': float,
                    'DTI_R': float, 'LTV_R': float, 'FIRST_PAYMENT_DATE': str, 'BAL_REPAID': float, 'T_ACT_12M': float, 'T_DEL_30D': float, 
                    'T_DEL_60D': float, 'LABEL': float, 'REMAINING_MONTHS_TO_LEGAL_MATURITY': float, "TIME_TO_EVENT": float, 'TOTAL_OBSERVED_LENGTH': float}


df_blumenstock = dd.read_parquet(data_folder + "./blumenstock_labeled_sample_orig_*.parquet.gzip")
df_blumenstock = df_blumenstock.astype(blumenstock_types)

In [5]:
from torch.utils.data import DataLoader
from FREDDIEMAC_baseline_data import FREDDIEMAC_basline_dataset, FREDDIEMAC_dataloader

BATCH_SIZE = 2**8

frac_cases=0.5
test_set = False
augment = False
data_augment_factor = 3
random_state = 42

allowed_covariates = ['INT_RATE', 'ORIG_UPB', 'FICO_SCORE', 'DTI_R','LTV_R', 'BAL_REPAID', 
                     'T_ACT_12M', 'T_DEL_30D', 'T_DEL_60D', 'REMAINING_MONTHS_TO_LEGAL_MATURITY']

TIME_TO_EVENT_covariate = 'TIME_TO_EVENT'
LABEL_covariate = 'LABEL'


FREDDIEMAC_raw_dataset = FREDDIEMAC_basline_dataset(df_blumenstock, 
                                                    allowed_covariates,
                                                    TIME_TO_EVENT_covariate,
                                                    LABEL_covariate,
                                                    frac_cases,
                                                    random_state,
                                                    test_set,
                                                    augment,
                                                    data_augment_factor)



data_loader = FREDDIEMAC_dataloader(dataset=FREDDIEMAC_raw_dataset,
                                    batch_size=BATCH_SIZE)

batch_data, batch_data_length, batch_event, batch_tte = next(iter(data_loader))

print(batch_data.shape)
print(batch_data_length.shape)
print(batch_event.shape)
print(batch_tte.shape)

torch.Size([256, 1, 10])
torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])


### Data Exploration

In [11]:
batch_data, batch_data_length, batch_event, batch_tte = next(iter(data_loader))

for i in range(min(BATCH_SIZE, 16)):
    print("batch event= %d --- batch_data_length= %d --- batch_tte= %d" % (batch_event[i], batch_data_length[i], batch_tte[i]))

batch event= 0 --- batch_data_length= 38 --- batch_tte= 38
batch event= 0 --- batch_data_length= 100 --- batch_tte= 100
batch event= 0 --- batch_data_length= 29 --- batch_tte= 29
batch event= 0 --- batch_data_length= 31 --- batch_tte= 31
batch event= 0 --- batch_data_length= 16 --- batch_tte= 16
batch event= 0 --- batch_data_length= 16 --- batch_tte= 16
batch event= 0 --- batch_data_length= 19 --- batch_tte= 19
batch event= 3 --- batch_data_length= 143 --- batch_tte= 143
batch event= 0 --- batch_data_length= 80 --- batch_tte= 80
batch event= 0 --- batch_data_length= 20 --- batch_tte= 20
batch event= 0 --- batch_data_length= 18 --- batch_tte= 18
batch event= 0 --- batch_data_length= 98 --- batch_tte= 98
batch event= 0 --- batch_data_length= 119 --- batch_tte= 119
batch event= 0 --- batch_data_length= 97 --- batch_tte= 97
batch event= 0 --- batch_data_length= 24 --- batch_tte= 24
batch event= 0 --- batch_data_length= 33 --- batch_tte= 33


# 2. Hyperparameters

In [ ]:
test_poc_raw_dataset = PocDataset(num_cases=1)
test_data_loader = DataLoader(dataset=test_poc_raw_dataset,batch_size=1,pin_memory=True)
test_batch_data, test_batch_data_length, test_batch_event, test_tte, test_meta = next(iter(test_data_loader))

# 3. Defining The Model